# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [15]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [16]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [17]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [18]:
# What model to download.
# MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
# MODEL_FILE = MODEL_NAME + '.tar.gz'
# DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
# PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_CKPT = './train_checkpoint_prefix_3/frozen_inference_graph.pb'
#train_checkpoint_prefix_3

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('train_checkpoint_prefix_2', 'disk_label_map.pbtxt')

NUM_CLASSES = 3

## Download Model

In [19]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [20]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [21]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [22]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [23]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
# PATH_TO_TEST_IMAGES_DIR = 'test_images'
# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
TEST_IMAGE_PATHS = ['./datas/wxtest/a1188a01-8185-3a77-9632-b2111b1bad7a.jpg']
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [25]:
#shunxu
sample_path = '/media/ss/38cfe914-26f2-4a22-9cf1-bea9684775ac/lmy/models-master/research/submit_sample.csv'
lines = open(sample_path).readlines()
TEST_IMAGE_PATHS = [os.path.join('./datas/wxtest', x.split(',')[0]) for x in lines]
obxes = []
obxes1 = []
with detection_graph.as_default():
  with tf.Session() as sess:
    for idd,image_path in enumerate(TEST_IMAGE_PATHS):
      image = Image.open(image_path)
      image_np = load_image_into_numpy_array(image)
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
      if idd<=413 and idd>=129:
        alfa = 0.1
        c = 'bridge'
        m = 10
      elif idd<129:
        alfa = 0.5
        c = 'airport'
        m=4
      else :
        alfa = 0.5
        c = 'harbor'
        m=4
      for i in range(m):
        if (i==0) or (output_dict['detection_scores'][i]>alfa and category_index[output_dict['detection_classes'][i]]['name']==c):
          box = output_dict['detection_boxes'][i].tolist()
          w,h,d = image_np.shape
          x0 = int(box[0]*w)
          x1 = int(box[2]*w)
          y0 = int(box[1]*h)
          y1 = int(box[3]*h)
          obx = [y0,x0,y1-y0,x1-x0]
          obx = ' '.join([str(x) for x in obx])
          obx = [os.path.basename(image_path), category_index[output_dict['detection_classes'][i]]['name'], obx]
          print(idd, i, obx)
          if i==0:
            obxes1.append(','.join(obx))
          obxes.append(','.join(obx))
line = '\n'.join(obxes)
import time
with open('sub_'+time.strftime('%m%d_%H%M', time.localtime(time.time()))+'.csv','w') as f:
    f.write(line)
line = '\n'.join(obxes1)
import time
with open('sub_'+time.strftime('%m%d_%H%M', time.localtime(time.time()))+'_1.csv','w') as f:
    f.write(line)

(0, 0, ['7fa7451d-6f3e-3d63-9eaf-1b513229a57a.jpg', u'airport', '329 359 501 776'])
(1, 0, ['098a9318-9f5a-3975-9005-07b860da7bbf.jpg', u'airport', '603 271 100 635'])
(2, 0, ['9fea314d-f306-3948-9010-a3a161f8c409.jpg', u'airport', '313 112 484 530'])
(3, 0, ['2fb950f5-ecf5-3ab2-95e1-fe396d1ea568.jpg', u'airport', '409 524 892 370'])
(4, 0, ['fa5b6a2d-dcaa-30a7-a49d-b4babe1b5f41.jpg', u'airport', '406 437 531 460'])
(5, 0, ['097f6b43-6c38-3234-b85c-65886ba36747.jpg', u'airport', '279 504 937 730'])
(6, 0, ['7cd1cd0d-e3b4-31a7-a9bd-a653794f9f2a.jpg', u'airport', '389 192 1425 653'])
(7, 0, ['29867bd8-c64b-3278-9e9d-6025e4c1c046.jpg', u'airport', '681 35 175 739'])
(8, 0, ['8d61e564-5d00-3c5f-80c6-1bc45e6c93e7.jpg', u'airport', '729 257 141 831'])
(9, 0, ['abbd598b-131f-33f9-b0d2-e7f16eb59344.jpg', u'airport', '366 315 603 525'])
(10, 0, ['d9ee38f0-a3a8-3581-91a1-3cb20a284daa.jpg', u'airport', '122 215 720 408'])
(11, 0, ['d2f70e08-f74f-3037-8e42-30a1fc40ff57.jpg', u'airport', '684 341 1

(79, 0, ['d9469861-f74d-3f37-acc0-47767e63f3de.jpg', u'bridge', '1223 366 46 45'])
(80, 0, ['a9b2755c-5e1b-3ac0-b566-9f3f7f8460c4.jpg', u'airport', '1294 1100 624 431'])
(81, 0, ['cc826c13-403f-3d69-8a1a-d8158c56ba78.jpg', u'airport', '3111 1278 1017 1760'])
(82, 0, ['0c7cea22-f450-3e10-aed6-18c3cd0a44ba.jpg', u'airport', '1709 380 491 751'])
(83, 0, ['fc1cfc0f-f969-3285-a297-46931f1d2320.jpg', u'airport', '742 920 966 990'])
(84, 0, ['c6b8eb96-cc17-3639-9368-f6b6e8773c12.jpg', u'airport', '229 165 1049 598'])
(85, 0, ['e8744fea-2b3b-31de-a96c-7d06ab5c0657.jpg', u'airport', '1371 361 965 832'])
(86, 0, ['353e7f3d-aea0-3ad4-b9c6-450275cb508f.jpg', u'airport', '3135 1551 1743 1166'])
(87, 0, ['2725a252-c565-3768-8ac8-8b5e77521c91.jpg', u'airport', '825 1351 1320 300'])
(88, 0, ['e358c6c6-7587-3204-9477-1ced7995b6c6.jpg', u'airport', '1389 431 212 892'])
(89, 0, ['a0afca40-c40f-3960-b667-1e6c257f1a0f.jpg', u'airport', '1792 517 2846 2466'])
(90, 0, ['c3dfb756-0b0a-396d-a38f-c87d0b70a630.j

(150, 0, ['1c392523-0771-35c6-a3ef-27b0f5f0478a.jpg', u'bridge', '1254 0 76 107'])
(150, 1, ['1c392523-0771-35c6-a3ef-27b0f5f0478a.jpg', u'bridge', '1291 280 67 142'])
(150, 2, ['1c392523-0771-35c6-a3ef-27b0f5f0478a.jpg', u'bridge', '738 262 87 251'])
(150, 3, ['1c392523-0771-35c6-a3ef-27b0f5f0478a.jpg', u'bridge', '1298 251 109 173'])
(150, 4, ['1c392523-0771-35c6-a3ef-27b0f5f0478a.jpg', u'bridge', '1351 239 59 175'])
(150, 5, ['1c392523-0771-35c6-a3ef-27b0f5f0478a.jpg', u'bridge', '1434 12 89 52'])
(151, 0, ['046030f9-deaa-3e1d-a551-38635749e3a6.jpg', u'bridge', '589 757 590 260'])
(152, 0, ['559c509d-628d-3e90-ac94-ed3a351c7b93.jpg', u'bridge', '462 477 136 68'])
(152, 1, ['559c509d-628d-3e90-ac94-ed3a351c7b93.jpg', u'bridge', '763 503 82 49'])
(152, 2, ['559c509d-628d-3e90-ac94-ed3a351c7b93.jpg', u'bridge', '874 506 56 29'])
(152, 3, ['559c509d-628d-3e90-ac94-ed3a351c7b93.jpg', u'bridge', '874 516 53 30'])
(152, 4, ['559c509d-628d-3e90-ac94-ed3a351c7b93.jpg', u'bridge', '784 492 77

(191, 0, ['30878b66-13a3-3b2c-886d-7206e08d1891.jpg', u'bridge', '757 345 49 53'])
(191, 1, ['30878b66-13a3-3b2c-886d-7206e08d1891.jpg', u'bridge', '581 293 55 71'])
(192, 0, ['6e5dbcdb-f6e1-3a22-9810-51fcfee32864.jpg', u'bridge', '674 237 24 39'])
(192, 1, ['6e5dbcdb-f6e1-3a22-9810-51fcfee32864.jpg', u'bridge', '677 373 81 41'])
(193, 0, ['979f3b4d-2644-37aa-9105-02ec263673dc.jpg', u'bridge', '674 237 24 39'])
(193, 1, ['979f3b4d-2644-37aa-9105-02ec263673dc.jpg', u'bridge', '677 373 81 41'])
(194, 0, ['b906ca9c-138b-3aa0-9839-9aa3e64f6c59.jpg', u'bridge', '674 237 24 39'])
(194, 1, ['b906ca9c-138b-3aa0-9839-9aa3e64f6c59.jpg', u'bridge', '677 373 81 41'])
(195, 0, ['d8674773-a501-3ee8-8763-c1d8548b6124.jpg', u'bridge', '674 237 24 39'])
(195, 1, ['d8674773-a501-3ee8-8763-c1d8548b6124.jpg', u'bridge', '677 373 81 41'])
(196, 0, ['f52b3713-b03e-39a5-8f2a-5e0c66b591a8.jpg', u'bridge', '674 237 24 39'])
(196, 1, ['f52b3713-b03e-39a5-8f2a-5e0c66b591a8.jpg', u'bridge', '677 373 81 41'])
(197

(226, 0, ['1a5374e7-8569-31a6-a5b7-c3f85b2495dd.jpg', u'bridge', '675 378 173 144'])
(226, 1, ['1a5374e7-8569-31a6-a5b7-c3f85b2495dd.jpg', u'bridge', '543 221 159 120'])
(226, 2, ['1a5374e7-8569-31a6-a5b7-c3f85b2495dd.jpg', u'bridge', '900 37 79 44'])
(226, 3, ['1a5374e7-8569-31a6-a5b7-c3f85b2495dd.jpg', u'bridge', '884 36 84 59'])
(226, 4, ['1a5374e7-8569-31a6-a5b7-c3f85b2495dd.jpg', u'bridge', '904 29 58 45'])
(226, 5, ['1a5374e7-8569-31a6-a5b7-c3f85b2495dd.jpg', u'bridge', '878 61 73 41'])
(226, 6, ['1a5374e7-8569-31a6-a5b7-c3f85b2495dd.jpg', u'bridge', '1447 8 74 77'])
(227, 0, ['0355ee79-3043-309c-9bfa-081a796a8b39.jpg', u'bridge', '675 378 173 144'])
(227, 1, ['0355ee79-3043-309c-9bfa-081a796a8b39.jpg', u'bridge', '543 221 159 120'])
(227, 2, ['0355ee79-3043-309c-9bfa-081a796a8b39.jpg', u'bridge', '900 37 79 44'])
(227, 3, ['0355ee79-3043-309c-9bfa-081a796a8b39.jpg', u'bridge', '884 36 84 59'])
(227, 4, ['0355ee79-3043-309c-9bfa-081a796a8b39.jpg', u'bridge', '904 29 58 45'])
(227

...

(414, 0, ['2fd1cb88-fb12-3b1d-bb93-d88d95b3a1fe.jpg', u'harbor', '192 79 455 416'])
(415, 0, ['7b400125-4e71-3970-a890-351b4df0cc41.jpg', u'harbor', '378 0 706 351'])
(415, 1, ['7b400125-4e71-3970-a890-351b4df0cc41.jpg', u'harbor', '1214 10 370 409'])
(416, 0, ['d5f65c29-2e32-3390-9009-1b34797c8ebd.jpg', u'harbor', '70 103 831 635'])
(416, 1, ['d5f65c29-2e32-3390-9009-1b34797c8ebd.jpg', u'harbor', '28 313 680 432'])
(416, 2, ['d5f65c29-2e32-3390-9009-1b34797c8ebd.jpg', u'harbor', '1190 2 352 210'])
(416, 3, ['d5f65c29-2e32-3390-9009-1b34797c8ebd.jpg', u'harbor', '243 124 870 657'])
(417, 0, ['cbc7e8e3-6552-3342-8b7a-c7e73ecb4cac.jpg', u'harbor', '70 103 831 635'])
(417, 1, ['cbc7e8e3-6552-3342-8b7a-c7e73ecb4cac.jpg', u'harbor', '28 313 680 432'])
(417, 2, ['cbc7e8e3-6552-3342-8b7a-c7e73ecb4cac.jpg', u'harbor', '1190 2 352 210'])
(417, 3, ['cbc7e8e3-6552-3342-8b7a-c7e73ecb4cac.jpg', u'harbor', '243 124 870 657'])
(418, 0, ['8633d7f2-eec4-34f3-92d6-bc66e3aa5e22.jpg', u'harbor', '361 160 

(463, 0, ['9dbc5149-7774-30f0-b49a-bca1783247c4.jpg', u'harbor', '672 66 752 698'])
(463, 1, ['9dbc5149-7774-30f0-b49a-bca1783247c4.jpg', u'harbor', '1014 111 551 623'])
(464, 0, ['1dc6c787-7eee-3e0c-bb5a-1448e7d472d9.jpg', u'harbor', '254 0 987 381'])
(464, 1, ['1dc6c787-7eee-3e0c-bb5a-1448e7d472d9.jpg', u'harbor', '207 3 789 283'])
(464, 2, ['1dc6c787-7eee-3e0c-bb5a-1448e7d472d9.jpg', u'harbor', '695 0 576 429'])
(465, 0, ['1d7a9417-ff19-3d09-a2b2-a6b887f3efd4.jpg', u'harbor', '254 0 987 381'])
(465, 1, ['1d7a9417-ff19-3d09-a2b2-a6b887f3efd4.jpg', u'harbor', '207 3 789 283'])
(465, 2, ['1d7a9417-ff19-3d09-a2b2-a6b887f3efd4.jpg', u'harbor', '695 0 576 429'])
(466, 0, ['5c853d1b-b459-3da2-9822-d6a1af139ff7.jpg', u'harbor', '634 309 966 365'])
(466, 1, ['5c853d1b-b459-3da2-9822-d6a1af139ff7.jpg', u'harbor', '1034 312 605 471'])
(467, 0, ['e99333cd-b69b-3f37-a008-566d2310fbe4.jpg', u'harbor', '634 309 966 365'])
(467, 1, ['e99333cd-b69b-3f37-a008-566d2310fbe4.jpg', u'harbor', '1034 312 6

(506, 0, ['90c0d446-ee15-334c-ac41-5e6664254d3f.jpg', u'harbor', '106 177 891 562'])
(506, 1, ['90c0d446-ee15-334c-ac41-5e6664254d3f.jpg', u'harbor', '711 29 636 406'])
(506, 2, ['90c0d446-ee15-334c-ac41-5e6664254d3f.jpg', u'harbor', '90 324 663 425'])
(506, 3, ['90c0d446-ee15-334c-ac41-5e6664254d3f.jpg', u'harbor', '169 47 1116 672'])
(507, 0, ['832096ae-b3ee-3380-b979-d9d548dc7b02.jpg', u'harbor', '637 304 608 403'])
(507, 1, ['832096ae-b3ee-3380-b979-d9d548dc7b02.jpg', u'harbor', '37 464 734 367'])
(507, 2, ['832096ae-b3ee-3380-b979-d9d548dc7b02.jpg', u'harbor', '91 404 1102 426'])
(507, 3, ['832096ae-b3ee-3380-b979-d9d548dc7b02.jpg', u'harbor', '412 315 800 446'])
(508, 0, ['097bf3f9-7a5c-3f78-ac65-b2c10197aa9f.jpg', u'harbor', '637 304 608 403'])
(508, 1, ['097bf3f9-7a5c-3f78-ac65-b2c10197aa9f.jpg', u'harbor', '37 464 734 367'])
(508, 2, ['097bf3f9-7a5c-3f78-ac65-b2c10197aa9f.jpg', u'harbor', '91 404 1102 426'])
(508, 3, ['097bf3f9-7a5c-3f78-ac65-b2c10197aa9f.jpg', u'harbor', '412

(540, 0, ['81266d64-6e74-32f4-96ed-20a8cba13004.jpg', u'harbor', '470 293 676 439'])
(540, 1, ['81266d64-6e74-32f4-96ed-20a8cba13004.jpg', u'harbor', '65 420 460 320'])
(540, 2, ['81266d64-6e74-32f4-96ed-20a8cba13004.jpg', u'harbor', '78 298 1046 479'])
(541, 0, ['f1547f94-c06f-3c45-a064-b8c7f9814d6f.jpg', u'harbor', '563 128 925 513'])
(541, 1, ['f1547f94-c06f-3c45-a064-b8c7f9814d6f.jpg', u'harbor', '529 236 775 431'])
(541, 2, ['f1547f94-c06f-3c45-a064-b8c7f9814d6f.jpg', u'harbor', '797 117 753 432'])
(542, 0, ['5a52a401-98c8-38d2-8ea8-33d4ae206fb3.jpg', u'harbor', '563 128 925 513'])
(542, 1, ['5a52a401-98c8-38d2-8ea8-33d4ae206fb3.jpg', u'harbor', '529 236 775 431'])
(542, 2, ['5a52a401-98c8-38d2-8ea8-33d4ae206fb3.jpg', u'harbor', '797 117 753 432'])
(543, 0, ['5b0ba483-759e-37d0-a893-fa25d3c6c1f8.jpg', u'harbor', '14 451 650 385'])
(543, 1, ['5b0ba483-759e-37d0-a893-fa25d3c6c1f8.jpg', u'harbor', '197 47 594 382'])
(544, 0, ['79a54af7-92be-310f-b722-3813e13372fe.jpg', u'harbor', '14

(586, 0, ['5f80e33d-c5f8-312c-a974-ef96364b3187.jpg', u'harbor', '203 242 1107 580'])
(586, 1, ['5f80e33d-c5f8-312c-a974-ef96364b3187.jpg', u'harbor', '669 471 786 365'])
(586, 2, ['5f80e33d-c5f8-312c-a974-ef96364b3187.jpg', u'harbor', '192 247 661 536'])
(586, 3, ['5f80e33d-c5f8-312c-a974-ef96364b3187.jpg', u'harbor', '480 347 950 474'])
(587, 0, ['dcc2b8a5-8beb-3e74-84de-9487892c578d.jpg', u'harbor', '29 87 573 416'])
(587, 1, ['dcc2b8a5-8beb-3e74-84de-9487892c578d.jpg', u'harbor', '1314 499 337 348'])
(587, 2, ['dcc2b8a5-8beb-3e74-84de-9487892c578d.jpg', u'harbor', '1168 0 484 524'])
(587, 3, ['dcc2b8a5-8beb-3e74-84de-9487892c578d.jpg', u'harbor', '145 87 1223 481'])
(588, 0, ['a1188a01-8185-3a77-9632-b2111b1bad7a.jpg', u'harbor', '29 87 573 416'])
(588, 1, ['a1188a01-8185-3a77-9632-b2111b1bad7a.jpg', u'harbor', '1314 499 337 348'])
(588, 2, ['a1188a01-8185-3a77-9632-b2111b1bad7a.jpg', u'harbor', '1168 0 484 524'])
(588, 3, ['a1188a01-8185-3a77-9632-b2111b1bad7a.jpg', u'harbor', '14

In [ ]:
import glob
TEST_IMAGE_PATHS = glob.glob(os.path.join('/media/ss/38cfe914-26f2-4a22-9cf1-bea9684775ac/lmy/models-master/research/object_detection/datas/wxtest','*.jpg'))
TEST_IMAGE_PATHS = sorted(TEST_IMAGE_PATHS)
obxes = []
obxes1 = []
with detection_graph.as_default():
  with tf.Session() as sess:
    for idd,image_path in enumerate(TEST_IMAGE_PATHS):
      image = Image.open(image_path)
      image_np = load_image_into_numpy_array(image)
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
      for i in range(4):
        if (i==0 and output_dict['detection_scores'][i]>0.5) or output_dict['detection_scores'][i]>0.5:
          box = output_dict['detection_boxes'][i].tolist()
          w,h,d = image_np.shape
          x0 = int(box[0]*w)
          x1 = int(box[2]*w)
          y0 = int(box[1]*h)
          y1 = int(box[3]*h)
          obx = [y0,x0,y1-y0,x1-x0]
          obx = ' '.join([str(x) for x in obx])
          obx = [os.path.basename(image_path), category_index[output_dict['detection_classes'][i]]['name'], obx]
          print(idd, i, obx)
          if i==0:
            obxes1.append(','.join(obx))
          obxes.append(','.join(obx))
line = '\n'.join(obxes)
import time
with open('sub_'+time.strftime('%m%d_%H%M', time.localtime(time.time()))+'.csv','w') as f:
    f.write(line)
line = '\n'.join(obxes1)
import time
with open('sub_'+time.strftime('%m%d_%H%M', time.localtime(time.time()))+'_1.csv','w') as f:
    f.write(line)

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
import glob
for image_path in glob.glob('./datas/wxtest/0ae2134*.jpg'):#TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      #min_score_thresh=0,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

In [ ]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
#          min_score_thresh=0.83,
          agnostic_mode=False,
          line_thickness=3)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)
      #cv.imwrite('tiaoshui.jpg',image_np)

In [ ]:
import glob
TEST_IMAGE_PATHS = glob.glob(os.path.join('/media/ss/38cfe914-26f2-4a22-9cf1-bea9684775ac/lmy/models-master/research/object_detection/datas/wxtest','*.jpg'))

In [ ]:
len(obxes)

In [ ]:
line = '\n'.join(obxes)

In [ ]:
with open('sub_filterssd.csv','w') as f:
    f.write(line)

In [ ]:
output_dict['detection_scores'][:10]

In [ ]:
TEST_IMAGE_PATHS[414]

In [ ]:
sample_path = '/media/ss/38cfe914-26f2-4a22-9cf1-bea9684775ac/lmy/models-master/research/submit_sample.csv'
lines = open(sample_path).readlines()
TEST_IMAGE_PATHS = [os.path.join('./datas/wxtest', x.split(',')[0]) for x in lines]

In [ ]:
len(TEST_IMAGE_PATHS)

In [ ]:
TEST_IMAGE_PATHS[413]